In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from huggingface_hub import login
import os

In [2]:
with open("keys.txt") as f:
    os.environ["HF_TOKEN"] = f.read().strip()

# login using env var
login(os.environ["HF_TOKEN"])

print(f"Login Huggingface so that we can access the model")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Login Huggingface so that we can access the model


In [ ]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"

# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"   # or a biomedical instruct variant

# model_id = "ContactDoctor/Bio-Medical-Llama-3-8B"



In [63]:

pipe = pipeline("text-generation", model=model_id ,return_full_text=False, device_map="auto")

Device set to use cuda:0


In [ ]:
document = "phenylephrine comes as a tablet, a liquid, or a dissolving strip to take by mouth. it is usually taken every 4 hours as needed. follow the directions on your prescription label or the package label carefully, and ask your doctor or pharmacist to explain any part you do not understand. take phenylephrine exactly as directed. do not take more or less of it or take it more often than prescribed by your doctor or directed on the label.phenylephrine comes alone and in combination with other medications. ask your doctor or pharmacist for advice on which product is best for your symptoms. check nonprescription cough and cold product labels carefully before using two or more products at the same time. these products may contain the same active ingredient(s) and taking them together could cause you to receive an overdose. this is especially important if you will be giving cough and cold medications to a child.nonprescription cough and cold combination products, including products that contain phenylephrine, can cause serious side effects or death in young children. do not give these products to children younger than 4 years of age. if you give these products to children 4 to 11 years of age, use caution and follow the package directions carefully.if you are giving phenylephrine or a combination product that contains phenylephrine to a child, read the package label carefully to be sure that it is the right product for a child of that age. do not give phenylephrine products that are made for adults to children.before you give a phenylephrine product to a child, check the package label to find out how much medication the child should receive. give the dose that matches the child's age on the chart. ask the child's doctor if you don't know how much medication to give the child.if you are taking the liquid, do not use a household spoon to measure your dose. use the measuring spoon or cup that came with the medication or use a spoon made especially for measuring medication.if your symptoms do not get better within 7 days or if you have a fever, stop taking phenylephrine and call your doctor.if you are taking the dissolving strips, place one strip on your tongue and allow it to dissolve.about Phenylephrine"

prompt = ("Read the document and generate 5 clinically relevant questions.\n "    
    "Requirements:\n"   
    "1) Don't need answer. \n"
    "2) Output JSON string like this: {\"questions:\": ['question1','question2',...,'question5']}.\n\n"
    "3) Only put 5 questions.\n\n"
    f"Document:\n{document}\n\n"
    "Return only JSON array.")
   

In [68]:
out = pipe(prompt, max_new_tokens=512, temperature=0.4)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [69]:
print(out)

 Here is the document:
```
{
  "questions": [
    "What is phenylephrine?",
    "How is phenylephrine taken?",
    "What are the side effects of phenylephrine?",
    "How long does it take for phenylephrine to work?",
    "What are the contraindications of phenylephrine?"
  ]
}
```


In [ ]:
tok = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
gen = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tok,
    return_full_text=False,
    #max_new_tokens=300,
    )

Device set to use cuda:0


In [ ]:
document = "phenylephrine comes as a tablet, a liquid, or a dissolving strip to take by mouth. it is usually taken every 4 hours as needed. follow the directions on your prescription label or the package label carefully, and ask your doctor or pharmacist to explain any part you do not understand. take phenylephrine exactly as directed. do not take more or less of it or take it more often than prescribed by your doctor or directed on the label.phenylephrine comes alone and in combination with other medications. ask your doctor or pharmacist for advice on which product is best for your symptoms. check nonprescription cough and cold product labels carefully before using two or more products at the same time. these products may contain the same active ingredient(s) and taking them together could cause you to receive an overdose. this is especially important if you will be giving cough and cold medications to a child.nonprescription cough and cold combination products, including products that contain phenylephrine, can cause serious side effects or death in young children. do not give these products to children younger than 4 years of age. if you give these products to children 4 to 11 years of age, use caution and follow the package directions carefully.if you are giving phenylephrine or a combination product that contains phenylephrine to a child, read the package label carefully to be sure that it is the right product for a child of that age. do not give phenylephrine products that are made for adults to children.before you give a phenylephrine product to a child, check the package label to find out how much medication the child should receive. give the dose that matches the child's age on the chart. ask the child's doctor if you don't know how much medication to give the child.if you are taking the liquid, do not use a household spoon to measure your dose. use the measuring spoon or cup that came with the medication or use a spoon made especially for measuring medication.if your symptoms do not get better within 7 days or if you have a fever, stop taking phenylephrine and call your doctor.if you are taking the dissolving strips, place one strip on your tongue and allow it to dissolve.about Phenylephrine"

messages = [
  {"role": "system", "content": "You are a medical student."},
  {"role": "user", "content": (
    "Read the document and generate 5 clinically relevant questions.\n"    
    "Requirements:\n"    
    "1) Don't need answer. \n"
    "2) Output complete JSON string: {'questions:': ['question1','question2',...,'question5']}.\n"
    f"Document:\n{document}\n\n"
    "Return only JSON."
  )}
]
prompt = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

resp = gen(prompt, max_new_tokens=500, temperature=0.4,
           eos_token_id=tok.eos_token_id)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [72]:
print(resp)

{
  "questions": [
    "What is the recommended dosage of phenylephrine for adults?",
    "How often should phenylephrine be taken by mouth?",
    "What are the potential side effects of taking phenylephrine with other medications?",
    "When should phenylephrine not be given to children?",
    "How do I measure my dose of phenylephrine?"
  ]


### Now Try stanford-crfm/BioMedLM

In [3]:
model_id = "stanford-crfm/BioMedLM"

In [4]:
pipe = pipeline("text-generation", model=model_id ,return_full_text=False, device_map="auto")

Device set to use cuda:0


In [ ]:
document = "phenylephrine comes as a tablet, a liquid, or a dissolving strip to take by mouth. it is usually taken every 4 hours as needed. follow the directions on your prescription label or the package label carefully, and ask your doctor or pharmacist to explain any part you do not understand. take phenylephrine exactly as directed. do not take more or less of it or take it more often than prescribed by your doctor or directed on the label.phenylephrine comes alone and in combination with other medications. ask your doctor or pharmacist for advice on which product is best for your symptoms. check nonprescription cough and cold product labels carefully before using two or more products at the same time. these products may contain the same active ingredient(s) and taking them together could cause you to receive an overdose. this is especially important if you will be giving cough and cold medications to a child.nonprescription cough and cold combination products, including products that contain phenylephrine, can cause serious side effects or death in young children. do not give these products to children younger than 4 years of age. if you give these products to children 4 to 11 years of age, use caution and follow the package directions carefully.if you are giving phenylephrine or a combination product that contains phenylephrine to a child, read the package label carefully to be sure that it is the right product for a child of that age. do not give phenylephrine products that are made for adults to children.before you give a phenylephrine product to a child, check the package label to find out how much medication the child should receive. give the dose that matches the child's age on the chart. ask the child's doctor if you don't know how much medication to give the child.if you are taking the liquid, do not use a household spoon to measure your dose. use the measuring spoon or cup that came with the medication or use a spoon made especially for measuring medication.if your symptoms do not get better within 7 days or if you have a fever, stop taking phenylephrine and call your doctor.if you are taking the dissolving strips, place one strip on your tongue and allow it to dissolve.about Phenylephrine"

prompt = ("Read the document and generate 5 clinically relevant questions.\n"    
    "Requirements:\n"   
    "1) Don't need answer. \n"
    "2) Output JSON string like this: {\"questions:\": ['question1','question2',...,'question5']}.\n\n"
    "3) Only put 5 questions.\n\n"
    f"Document:\n{document}\n\n"
    "Return only JSON array.")

In [6]:
out = pipe(prompt, max_new_tokens=512, temperature=0.4)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (1024). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.
/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gather_kernel: block: [1024,0,0], thread: [224,0,0] Assertion `ind >=0 && ind < ind_dim_size && "vectorized gather kernel index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gather_kernel: block: [1024,0,0], thread: [225,0,0] Assertion `ind >=0 && ind < ind_dim_size && "vectorized gather kernel index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKernelUtils.cu:16: vectorized_gather_kernel: block: [1024,0,0], thread: [226,0,0] Assertion `ind >=0 && ind < ind_dim_size && "vectorized gather kernel index out of bounds"` failed.
/pytorch/aten/src/ATen/native/cuda/IndexKerne

AcceleratorError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [3]:
model_id = "ContactDoctor/Bio-Medical-Llama-3-8B"

In [4]:
pipe = pipeline("text-generation", model=model_id ,return_full_text=False, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


In [ ]:
document = "phenylephrine comes as a tablet, a liquid, or a dissolving strip to take by mouth. it is usually taken every 4 hours as needed. follow the directions on your prescription label or the package label carefully, and ask your doctor or pharmacist to explain any part you do not understand. take phenylephrine exactly as directed. do not take more or less of it or take it more often than prescribed by your doctor or directed on the label.phenylephrine comes alone and in combination with other medications. ask your doctor or pharmacist for advice on which product is best for your symptoms. check nonprescription cough and cold product labels carefully before using two or more products at the same time. these products may contain the same active ingredient(s) and taking them together could cause you to receive an overdose. this is especially important if you will be giving cough and cold medications to a child.nonprescription cough and cold combination products, including products that contain phenylephrine, can cause serious side effects or death in young children. do not give these products to children younger than 4 years of age. if you give these products to children 4 to 11 years of age, use caution and follow the package directions carefully.if you are giving phenylephrine or a combination product that contains phenylephrine to a child, read the package label carefully to be sure that it is the right product for a child of that age. do not give phenylephrine products that are made for adults to children.before you give a phenylephrine product to a child, check the package label to find out how much medication the child should receive. give the dose that matches the child's age on the chart. ask the child's doctor if you don't know how much medication to give the child.if you are taking the liquid, do not use a household spoon to measure your dose. use the measuring spoon or cup that came with the medication or use a spoon made especially for measuring medication.if your symptoms do not get better within 7 days or if you have a fever, stop taking phenylephrine and call your doctor.if you are taking the dissolving strips, place one strip on your tongue and allow it to dissolve.about Phenylephrine"

prompt = ("Read the document and generate 5 clinically relevant questions.\n"    
    "Requirements:\n"   
    #"1) Don't need answer. \n"
    "2) Output JSON string like this: {\"questions:\": ['','',...,'']}.\n\n"
    #"3) Only put 5 questions.\n\n"
    f"Document:\n{document}\n\n"
    "Return only JSON string without any words around the json string.")

In [16]:
out = pipe(prompt, max_new_tokens=512, temperature=0.4)[0]["generated_text"]

In [17]:
print(out)

{"questions": ["What is the typical dosing frequency of phenylephrine?", "Is it safe to take phenylephrine with other cough and cold medications?", "Can phenylephrine be given to children younger than 4 years of age?", "How do I determine the appropriate dose of phenylephrine for a child?", "What should I do if I experience no relief from phenylephrine after 7 days?"]}


In [18]:
out

'{"questions": ["What is the typical dosing frequency of phenylephrine?", "Is it safe to take phenylephrine with other cough and cold medications?", "Can phenylephrine be given to children younger than 4 years of age?", "How do I determine the appropriate dose of phenylephrine for a child?", "What should I do if I experience no relief from phenylephrine after 7 days?"]}'

In [ ]:
tok = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
gen = pipeline("text-generation", model=model, tokenizer=tok,return_full_text=False)

/home/jovyan/miniconda3/envs/ai311/lib/python3.11/site-packages/accelerate/utils/modeling.py:1582: UserWarning: Current model requires 256 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
document = "phenylephrine comes as a tablet, a liquid, or a dissolving strip to take by mouth. it is usually taken every 4 hours as needed. follow the directions on your prescription label or the package label carefully, and ask your doctor or pharmacist to explain any part you do not understand. take phenylephrine exactly as directed. do not take more or less of it or take it more often than prescribed by your doctor or directed on the label.phenylephrine comes alone and in combination with other medications. ask your doctor or pharmacist for advice on which product is best for your symptoms. check nonprescription cough and cold product labels carefully before using two or more products at the same time. these products may contain the same active ingredient(s) and taking them together could cause you to receive an overdose. this is especially important if you will be giving cough and cold medications to a child.nonprescription cough and cold combination products, including products that contain phenylephrine, can cause serious side effects or death in young children. do not give these products to children younger than 4 years of age. if you give these products to children 4 to 11 years of age, use caution and follow the package directions carefully.if you are giving phenylephrine or a combination product that contains phenylephrine to a child, read the package label carefully to be sure that it is the right product for a child of that age. do not give phenylephrine products that are made for adults to children.before you give a phenylephrine product to a child, check the package label to find out how much medication the child should receive. give the dose that matches the child's age on the chart. ask the child's doctor if you don't know how much medication to give the child.if you are taking the liquid, do not use a household spoon to measure your dose. use the measuring spoon or cup that came with the medication or use a spoon made especially for measuring medication.if your symptoms do not get better within 7 days or if you have a fever, stop taking phenylephrine and call your doctor.if you are taking the dissolving strips, place one strip on your tongue and allow it to dissolve.about Phenylephrine"

messages = [
  {"role": "system", "content": "You are a medical student."},
  {"role": "user", "content": (
    "Read the document and generate 5 clinically relevant questions.\n"    
    "Requirements:\n"    
    #"1) Don't need answer. \n"
    "2) Output complete JSON string: {'questions:': ['question1','question2',...,'question5']}.\n"
    f"Document:\n{document}\n\n"
    "Return only JSON."
  )}
]
prompt = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

resp = gen(prompt, max_new_tokens=500, temperature=0.4,
           eos_token_id=tok.eos_token_id)[0]["generated_text"]

In [ ]:
print(resp)

In [3]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct" 

In [4]:
pipe = pipeline("text-generation", model=model_id ,return_full_text=False, device_map="auto")

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Some parameters are on the meta device because they were offloaded to the cpu.


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Device set to use cuda:0


In [17]:
document = "phenylephrine comes as a tablet, a liquid, or a dissolving strip to take by mouth. it is usually taken every 4 hours as needed. follow the directions on your prescription label or the package label carefully, and ask your doctor or pharmacist to explain any part you do not understand. take phenylephrine exactly as directed. do not take more or less of it or take it more often than prescribed by your doctor or directed on the label.phenylephrine comes alone and in combination with other medications. ask your doctor or pharmacist for advice on which product is best for your symptoms. check nonprescription cough and cold product labels carefully before using two or more products at the same time. these products may contain the same active ingredient(s) and taking them together could cause you to receive an overdose. this is especially important if you will be giving cough and cold medications to a child.nonprescription cough and cold combination products, including products that contain phenylephrine, can cause serious side effects or death in young children. do not give these products to children younger than 4 years of age. if you give these products to children 4 to 11 years of age, use caution and follow the package directions carefully.if you are giving phenylephrine or a combination product that contains phenylephrine to a child, read the package label carefully to be sure that it is the right product for a child of that age. do not give phenylephrine products that are made for adults to children.before you give a phenylephrine product to a child, check the package label to find out how much medication the child should receive. give the dose that matches the child's age on the chart. ask the child's doctor if you don't know how much medication to give the child.if you are taking the liquid, do not use a household spoon to measure your dose. use the measuring spoon or cup that came with the medication or use a spoon made especially for measuring medication.if your symptoms do not get better within 7 days or if you have a fever, stop taking phenylephrine and call your doctor.if you are taking the dissolving strips, place one strip on your tongue and allow it to dissolve.about Phenylephrine"

prompt = ("Read the document Below and generate 5 clinically relevant questions. \n"    
    "Requirements:\n"   
    "1) Don't need python code at all. \n"
    "2) Output JSON string: {\"questions:\": ['','',...,'']}.\n\n"
    "3) Output only JSON string. \n"
    "4) Don't generate any other text besides the JSON string. \n"
    "5) Don't put Note after the JSON string. \n"
    #"3) Only put 5 questions.\n\n"
    f"Document:\n{document}\n\n"
    "Return only JSON without any words around the json object.")

In [18]:
out = pipe(prompt, max_new_tokens=512, temperature=0.4)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [19]:
print(out)

 
{"questions": ["What is the recommended frequency of taking phenylephrine?", "What should you do if you are taking phenylephrine and your symptoms do not get better within 7 days?", "What is the recommended age range for giving phenylephrine to children?", "What should you do if you are giving phenylephrine to a child and you are unsure about the dose?", "What is the recommended way to measure the dose of phenylephrine liquid?"]} 





```
{"questions": ["What is the recommended frequency of taking phenylephrine?", "What should you do if you are taking phenylephrine and your symptoms do not get better within 7 days?", "What is the recommended age range for giving phenylephrine to children?", "What should you do if you are giving phenylephrine to a child and you are unsure about the dose?", "What is the recommended way to measure the dose of phenylephrine liquid?"]}
``` 





```
{"questions": ["What is the recommended frequency of taking phenylephrine?", "What should you do if you ar

### test do summarization

In [3]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"

In [4]:
pipe = pipeline("text-generation", model=model_id ,return_full_text=False, device_map="auto")

Device set to use cuda:0


In [5]:
document = "phenylephrine comes as a tablet, a liquid, or a dissolving strip to take by mouth. it is usually taken every 4 hours as needed. follow the directions on your prescription label or the package label carefully, and ask your doctor or pharmacist to explain any part you do not understand. take phenylephrine exactly as directed. do not take more or less of it or take it more often than prescribed by your doctor or directed on the label.phenylephrine comes alone and in combination with other medications. ask your doctor or pharmacist for advice on which product is best for your symptoms. check nonprescription cough and cold product labels carefully before using two or more products at the same time. these products may contain the same active ingredient(s) and taking them together could cause you to receive an overdose. this is especially important if you will be giving cough and cold medications to a child.nonprescription cough and cold combination products, including products that contain phenylephrine, can cause serious side effects or death in young children. do not give these products to children younger than 4 years of age. if you give these products to children 4 to 11 years of age, use caution and follow the package directions carefully.if you are giving phenylephrine or a combination product that contains phenylephrine to a child, read the package label carefully to be sure that it is the right product for a child of that age. do not give phenylephrine products that are made for adults to children.before you give a phenylephrine product to a child, check the package label to find out how much medication the child should receive. give the dose that matches the child's age on the chart. ask the child's doctor if you don't know how much medication to give the child.if you are taking the liquid, do not use a household spoon to measure your dose. use the measuring spoon or cup that came with the medication or use a spoon made especially for measuring medication.if your symptoms do not get better within 7 days or if you have a fever, stop taking phenylephrine and call your doctor.if you are taking the dissolving strips, place one strip on your tongue and allow it to dissolve.about Phenylephrine"

prompt = ("Read the document and do a summarization.\n "    
    "Requirements:\n"   
    "1) Only give me a summarization under 300 words. \n"
    "2) The summarization only have one paragraph\n"    
    f"Document:\n{document}\n\n"
    "Return summarization.")

In [6]:
out = pipe(prompt, max_new_tokens=512, temperature=0.4)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [7]:
print(out)

 

Phenylephrine is a medication that comes in various forms, including tablets, liquids, and dissolving strips. It is usually taken every 4 hours as needed, and the exact dosage depends on the specific product and the individual's needs. Taking phenylephrine exactly as directed is crucial to avoid side effects and ensure effective treatment. It is essential to check the product label carefully before using multiple products at the same time and to consult with a doctor or pharmacist if you have any questions or concerns. Phenylephrine is available in combination with other medications, and it is recommended to use caution when giving it to children, especially those under 4 years old. It is also important to follow the package directions carefully and to consult with a doctor if you are unsure about the correct dosage or the product's suitability for your child. In addition, phenylephrine can cause serious side effects or death in young children, so it is essential to use it only unde

In [8]:
model_id = "ContactDoctor/Bio-Medical-Llama-3-8B"

In [9]:
pipe = pipeline("text-generation", model=model_id ,return_full_text=False, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


In [10]:
document = "phenylephrine comes as a tablet, a liquid, or a dissolving strip to take by mouth. it is usually taken every 4 hours as needed. follow the directions on your prescription label or the package label carefully, and ask your doctor or pharmacist to explain any part you do not understand. take phenylephrine exactly as directed. do not take more or less of it or take it more often than prescribed by your doctor or directed on the label.phenylephrine comes alone and in combination with other medications. ask your doctor or pharmacist for advice on which product is best for your symptoms. check nonprescription cough and cold product labels carefully before using two or more products at the same time. these products may contain the same active ingredient(s) and taking them together could cause you to receive an overdose. this is especially important if you will be giving cough and cold medications to a child.nonprescription cough and cold combination products, including products that contain phenylephrine, can cause serious side effects or death in young children. do not give these products to children younger than 4 years of age. if you give these products to children 4 to 11 years of age, use caution and follow the package directions carefully.if you are giving phenylephrine or a combination product that contains phenylephrine to a child, read the package label carefully to be sure that it is the right product for a child of that age. do not give phenylephrine products that are made for adults to children.before you give a phenylephrine product to a child, check the package label to find out how much medication the child should receive. give the dose that matches the child's age on the chart. ask the child's doctor if you don't know how much medication to give the child.if you are taking the liquid, do not use a household spoon to measure your dose. use the measuring spoon or cup that came with the medication or use a spoon made especially for measuring medication.if your symptoms do not get better within 7 days or if you have a fever, stop taking phenylephrine and call your doctor.if you are taking the dissolving strips, place one strip on your tongue and allow it to dissolve.about Phenylephrine"

prompt = ("Read the document and do a summarization.\n "    
    "Requirements:\n"   
    "1) Only give me a summarization under 300 words. \n"
    "2) The summarization only have one paragraph\n"    
    f"Document:\n{document}\n\n"
    "Return summarization.")

In [11]:
out = pipe(prompt, max_new_tokens=512, temperature=0.4)[0]["generated_text"]

In [12]:
print(out)

 
Phenylephrine is a medication used to treat nasal congestion and eye redness. It comes in various forms such as tablets, liquids, and dissolving strips, and should be taken exactly as directed by a doctor or the label. It is important to carefully read the label and follow the instructions, especially when giving it to children. The medication should not be given to children under the age of four, and proper dosage should be followed for children between the ages of four and eleven. Additionally, phenylephrine should not be used with other cough and cold medications without consulting a doctor, as this can lead to an overdose. If symptoms persist or worsen, one should stop taking the medication and consult a doctor.


In [15]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct" 

In [16]:
pipe = pipeline("text-generation", model=model_id ,return_full_text=False, device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


In [17]:
document = "phenylephrine comes as a tablet, a liquid, or a dissolving strip to take by mouth. it is usually taken every 4 hours as needed. follow the directions on your prescription label or the package label carefully, and ask your doctor or pharmacist to explain any part you do not understand. take phenylephrine exactly as directed. do not take more or less of it or take it more often than prescribed by your doctor or directed on the label.phenylephrine comes alone and in combination with other medications. ask your doctor or pharmacist for advice on which product is best for your symptoms. check nonprescription cough and cold product labels carefully before using two or more products at the same time. these products may contain the same active ingredient(s) and taking them together could cause you to receive an overdose. this is especially important if you will be giving cough and cold medications to a child.nonprescription cough and cold combination products, including products that contain phenylephrine, can cause serious side effects or death in young children. do not give these products to children younger than 4 years of age. if you give these products to children 4 to 11 years of age, use caution and follow the package directions carefully.if you are giving phenylephrine or a combination product that contains phenylephrine to a child, read the package label carefully to be sure that it is the right product for a child of that age. do not give phenylephrine products that are made for adults to children.before you give a phenylephrine product to a child, check the package label to find out how much medication the child should receive. give the dose that matches the child's age on the chart. ask the child's doctor if you don't know how much medication to give the child.if you are taking the liquid, do not use a household spoon to measure your dose. use the measuring spoon or cup that came with the medication or use a spoon made especially for measuring medication.if your symptoms do not get better within 7 days or if you have a fever, stop taking phenylephrine and call your doctor.if you are taking the dissolving strips, place one strip on your tongue and allow it to dissolve.about Phenylephrine"

prompt = ("Read the document and do a summarization.\n "    
    "Requirements:\n"   
    "1) Only give me a summarization under 300 words. \n"
    "2) The summarization only have one paragraph\n"    
    f"Document:\n{document}\n\n"
    "Return summarization.")

In [ ]:
out = pipe(prompt, max_new_tokens=512, temperature=0.4)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(out)

In [3]:
model_id = "meta-llama/Llama-3.2-3B" 

In [4]:
pipe = pipeline("text-generation", model=model_id ,return_full_text=False, device_map="auto")

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Device set to use cuda:0


In [20]:
document = "phenylephrine comes as a tablet, a liquid, or a dissolving strip to take by mouth. it is usually taken every 4 hours as needed. follow the directions on your prescription label or the package label carefully, and ask your doctor or pharmacist to explain any part you do not understand. take phenylephrine exactly as directed. do not take more or less of it or take it more often than prescribed by your doctor or directed on the label.phenylephrine comes alone and in combination with other medications. ask your doctor or pharmacist for advice on which product is best for your symptoms. check nonprescription cough and cold product labels carefully before using two or more products at the same time. these products may contain the same active ingredient(s) and taking them together could cause you to receive an overdose. this is especially important if you will be giving cough and cold medications to a child.nonprescription cough and cold combination products, including products that contain phenylephrine, can cause serious side effects or death in young children. do not give these products to children younger than 4 years of age. if you give these products to children 4 to 11 years of age, use caution and follow the package directions carefully.if you are giving phenylephrine or a combination product that contains phenylephrine to a child, read the package label carefully to be sure that it is the right product for a child of that age. do not give phenylephrine products that are made for adults to children.before you give a phenylephrine product to a child, check the package label to find out how much medication the child should receive. give the dose that matches the child's age on the chart. ask the child's doctor if you don't know how much medication to give the child.if you are taking the liquid, do not use a household spoon to measure your dose. use the measuring spoon or cup that came with the medication or use a spoon made especially for measuring medication.if your symptoms do not get better within 7 days or if you have a fever, stop taking phenylephrine and call your doctor.if you are taking the dissolving strips, place one strip on your tongue and allow it to dissolve.about Phenylephrine"

prompt = ("Please give me a summarization of the document below.\n "   
    "The summarization only contains under 300 words.\n"
    "The summarization only has one paragraph.\n"
    f"Document:\n{document}\n\n"
    )

In [21]:
out = pipe(prompt, max_new_tokens=512, temperature=0.4)[0]["generated_text"]

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [22]:
print(out)

Phenylephrine is used to treat stuffy nose, sinus congestion, and runny nose caused by the common cold, flu, or other breathing illnesses (e.g., hay fever). Phenylephrine is also used to treat nasal congestion caused by a cold, flu, hay fever, or other breathing illnesses. It is also used to treat nasal congestion caused by an enlarged prostate (benign prostatic hyperplasia). Phenylephrine is a decongestant that works by narrowing the blood vessels in the nasal passages. This helps to decrease congestion and stuffiness in the nose. Phenylephrine is also used to treat low blood pressure (hypotension) after surgery or to prevent low blood pressure caused by certain medications (e.g., antihistamines, tranquilizers). Phenylephrine is also used to treat low blood pressure (hypotension) that occurs during surgery or other procedures (e.g., spinal tap, regional anesthesia). Phenylephrine is also used to treat low blood pressure (hypotension) that occurs during or after surgery or other proced